In [1]:
%matplotlib inline

from time import time

## Load SDSS data and measure B/T and covariance

In [2]:
from empirical_disk_bulge.observations import load_umachine_sdss_complete

sdss = load_umachine_sdss_complete()
mask = sdss['type_mendel13'] != 4
mask *= sdss['deltaBD_mendel13'] <= 1
cut_sdss = sdss[mask]

from empirical_disk_bulge.observations import sfr_sequence_bulge_disk_fractions_vs_sm as bt_measurement
_result = bt_measurement(cut_sdss['bt'], cut_sdss['sm'], cut_sdss['ssfr'])
np.save('frac_bulge_disk_vs_sm_mendel13', np.array(_result))
sm_abscissa, frac_disk_dom_all, frac_bulge_dom_all,\
    frac_disk_dom_sfs, frac_bulge_dom_sfs, \
    frac_disk_dom_gv, frac_bulge_dom_gv, \
    frac_disk_dom_q, frac_bulge_dom_q = _result

sdss_data_vector = bt_measurement(cut_sdss['bt'], cut_sdss['sm'], cut_sdss['ssfr'], return_data_vector=True)
sdss_invcov = np.diag(0.2*sdss_data_vector)


In [3]:
from empirical_disk_bulge.models import random_constant_disruption

def model_prediction(x, sm, ssfr, smh, sfrh, tarr):
    prob_disrupt = min(1, max(0, x[0]))
    frac_disrupt = min(1, max(0, x[1]))
    zobs = 0.
    sm_disk, sm_bulge = random_constant_disruption(sfrh, smh, tarr, zobs, 
                                                   prob_disrupt, frac_disrupt)
    bt = sm_bulge/(sm_disk + sm_bulge)
    
    return bt_measurement(bt, sm, ssfr, return_data_vector=True)

def lnprior(x):
    prob_disrupt, frac_disrupt = x
    if (0 <= prob_disrupt <= 1) and (0 <= frac_disrupt <= 1):
        return 0.0
    else:
        return -np.inf

def lnlike(x, observations, icov, sm, ssfr, smh, sfrh, tarr):
    predictions = model_prediction(x, sm, ssfr, smh, sfrh, tarr)
    diff = predictions-observations
    return -np.dot(diff,np.dot(icov,diff))/2.0

def lnprob(x, observations, icov, sm, ssfr, smh, sfrh, tarr):
    prior = lnprior(x)
    if np.isinf(prior):
        return prior
    else:
        return lnlike(x, observations, icov, sm, ssfr, smh, sfrh, tarr) + prior

In [4]:
from umachine_pyio.load_mock import value_added_mock, load_mock_from_binaries
subvolumes = np.random.choice(np.arange(144), 50, replace=False)
vamock = value_added_mock(load_mock_from_binaries(subvolumes), 250)
sfrh, smh = vamock['sfr_history_main_prog'].data, vamock['sm_history_main_prog'].data
sm = np.log10(vamock['obs_sm'].data)
ssfr = np.log10(vamock['obs_sfr'].data/vamock['obs_sm'].data)

from umachine_pyio.load_mock import get_snapshot_times
cosmic_age_array = get_snapshot_times()

Total runtime = 9.86 seconds


In [5]:
x = np.array((0.5, 0.5))
%timeit lnprob(x, sdss_data_vector, sdss_invcov, sm, ssfr, smh, sfrh, cosmic_age_array)

1 loop, best of 3: 654 ms per loop


In [6]:
import emcee

args=[sdss_data_vector, sdss_invcov, sm, ssfr, smh, sfrh, cosmic_age_array]
ndim = 2

nwalkers = 4
p0 = np.random.rand(ndim * nwalkers).reshape((nwalkers, ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=args)

num_burnin = 15
start = time()
pos0, prob, state = sampler.run_mcmc(p0, num_burnin)
sampler.reset()
end = time()
print("Runtime for burn-in phase = {0:.2f} seconds".format(end-start))

Runtime for burn-in phase = 30.15 seconds


In [7]:
sep = "  "
formatter = sep.join("{"+str(i)+":.4f}" for i in range(pos0.shape[-1])) + "  " + "{"+str(pos0.shape[-1])+":.4f}\n"
header = "prob_disrupt  frac_disrupt  lnprob\n"

num_iterations = 50*40

start = time()

with open("chain.dat", "wb") as f:
    f.write(header)
    for result in sampler.sample(pos0, iterations=num_iterations, storechain=False):
        pos, prob, state = result
        for a, b in zip(pos, prob):
            newline = formatter.format(*np.append(a, b))
            f.write(newline)
end = time()
print("Runtime for MCMC = {0:.2f} minutes".format((end-start)/60.))

Runtime for MCMC = 73.25 minutes


In [9]:
print("Mean acceptance fraction: {0:.3f}".format(np.mean(sampler.acceptance_fraction)))

# import corner

# samples = sampler.chain[:, 10:, :].reshape((-1, ndim))
# fig = corner.corner(samples)

Mean acceptance fraction: 0.620


In [1]:
more chain.dat